# 📄 Local RAG PDF Chatbot with LangChain, HuggingFace, and Gradio

This notebook demonstrates how to build a fully local Retrieval-Augmented Generation (RAG) chatbot for querying your own PDF documents. It leverages LangChain, HuggingFace Transformers, FAISS for vector search, and Gradio for a user-friendly chat interface.

---

## **Workflow Overview**

1. **Install Required Libraries**  
    Install all necessary Python packages for document loading, embedding, vector storage, and the chat interface.

2. **Upload PDF Documents**  
    Upload your PDF files, which will be used as the knowledge base for the chatbot.

3. **Load and Process Documents**  
    - Load PDFs from the uploaded files.
    - Split documents into manageable text chunks.
    - Filter out empty or invalid chunks.

4. **Create Embeddings and Vector Store**  
    - Generate embeddings for each document chunk using a HuggingFace model.
    - Store embeddings in a FAISS vector database for efficient retrieval.

5. **Set Up Conversational RAG Chain**  
    - Initialize a lightweight HuggingFace language model for answer generation.
    - Set up a conversational retrieval chain using LangChain.

6. **Define Chatbot Logic**  
    - Implement a function to handle user queries, maintain chat history, and interact with the RAG chain.

7. **Launch Gradio Chat Interface**  
    - Provide a simple web-based chat interface for users to interact with the PDF chatbot.

8. **Automated Q&A and Export**  
    - Run a set of sample questions through the chatbot.
    - Save the resulting Q&A pairs to an Excel file for further analysis.

---

## **Cell-by-Cell Description**

### 1. Install Dependencies
Install all required libraries for document processing, embeddings, vector storage, and the chat interface.

### 2. Upload PDFs
Use Google Colab's file upload utility to upload your PDF documents. Move them to a dedicated `data` directory.

### 3. Install FAISS
Install the FAISS library for efficient vector similarity search.

### 4. Load and Process Documents
- Load all PDFs from the `data` directory.
- Split documents into overlapping text chunks.
- Filter out empty chunks.
- Generate embeddings using a HuggingFace model.
- Store embeddings in a FAISS vector store.

### 5. Set Up RAG Chain
- Initialize a lightweight HuggingFace language model.
- Create a retriever from the FAISS vector store.
- Set up a conversational retrieval chain using LangChain.

### 6. Define Chatbot Logic
- Implement a function to process user queries, maintain chat history, and interact with the RAG chain.
- Handle greetings and short queries gracefully.

### 7. Launch Gradio Interface
- Wrap the chatbot logic in a Gradio interface for interactive chatting.

### 8. Define Sample Questions
List a set of example questions to automatically query the chatbot.

### 9. Automated Q&A and Export
- Run the sample questions through the chatbot.
- Save the resulting Q&A pairs to an Excel file.

---

## **Usage**

1. **Run each cell in order.**
2. **Upload your PDF documents when prompted.**
3. **Interact with the chatbot via the Gradio interface.**
4. **Review and export Q&A results as needed.**

---

**This notebook provides a fully local, privacy-preserving solution for querying your own documents using state-of-the-art NLP techniques.**

In [1]:
!pip install -q \
    langchain-core langchain-community langchain-chroma \
    langchain-text-splitters sentence-transformers \
    transformers accelerate chromadb pypdf gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta

In [ ]:
from google.colab import files
uploaded = files.upload()

import os
os.makedirs("data", exist_ok=True)
for fname in uploaded.keys():
    os.rename(fname, f"data/{fname}")


Saving SDE_Resume (7).pdf to SDE_Resume (7).pdf


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
from uuid import uuid4

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS  # FAISS instead of Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain

from transformers import pipeline

# Set paths for data and FAISS index
# Note: In Colab, the default working directory is /content
DATA_PATH = "/content/data"
FAISS_INDEX_PATH = "/content/faiss_index"

# Clean up FAISS index directory
if os.path.exists(FAISS_INDEX_PATH):
    shutil.rmtree(FAISS_INDEX_PATH)
os.makedirs(DATA_PATH, exist_ok=True)

# Load PDFs
loader = PyPDFDirectoryLoader(DATA_PATH)
raw_documents = loader.load()

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=150)
chunks = splitter.split_documents(raw_documents)

# Filter valid chunks
valid_chunks = [chunk for chunk in chunks if chunk.page_content.strip()]

# Embedding model
embedding = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl")

# FAISS vector store
vector_store = FAISS.from_documents(valid_chunks, embedding)

vector_store.save_local(FAISS_INDEX_PATH)

<ipython-input-6-7a4a0ffa1826>:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain

# Lightweight model for RAG pipeline
# We can use OpenAI models, but they are not free
hf_pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=2048,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=hf_pipe)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

chat_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-9-fd61a963e033>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipe)


In [ ]:
#chat history as a list of tuples (question, answer)
chat_history = []

def ask_rag_bot(query: str) -> str:
    """
    Process a user query using the conversational RAG chain and maintain history.
    """

    query = query.strip()
    if not query:
        return "❓ Please enter a valid question."

    # Handle basic greetings
    greetings = ["hi", "hello", "hey"]
    if query.lower() in greetings:
        return "👋 Hi there! I’m your PDF assistant. Ask me anything based on the uploaded documents."

    if len(query.split()) < 3:
        return "🧐 Can you please provide more context in your question?"

    try:
        # Run the RAG pipeline
        result = chat_chain({
            "question": query,
            "chat_history": chat_history
        })

        answer = result.get("answer", "").strip()

        if not answer:
            return "🤔 I couldn't find an answer based on the documents."

        # Add to history only if valid
        chat_history.append((query, answer))

        return answer

    except Exception as e:
        return f"⚠️ An error occurred: {str(e)}"


In [11]:
import gradio as gr

# Define Gradio-friendly wrapper
def gradio_chat_interface(message, history):
    try:
        response = ask_rag_bot(message)
        return response
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# Launch interface
gr.ChatInterface(
    fn=gradio_chat_interface,
    title="📄 Document Chatbot (Free HuggingFace RAG)",
    description="Ask questions about your uploaded PDFs using a fully local RAG system.",
    textbox=gr.Textbox(placeholder="Ask something about your document..."),
).launch()


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d4a16afd39e0491d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
questions = [
    "What type of projects we worked on?",
    "Which college he attended?",
    "What B.Tech degree he is doing?",
    "What skills does he have?",
    "Which coding languages does he know?"
]


In [13]:
import pandas as pd

chat_history = []
qa_pairs = []

for q in questions:
    result = chat_chain({"question": q, "chat_history": chat_history})
    answer = result["answer"]
    qa_pairs.append((q, answer))
    chat_history.append((q, answer))

# Save to Excel
df = pd.DataFrame(qa_pairs, columns=["Question", "Answer"])
df.to_excel("RAG_QA_Output.xlsx", index=False)
print("✅ Saved to RAG_QA_Output.xlsx")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuratio

✅ Saved to RAG_QA_Output.xlsx
